<a href="https://colab.research.google.com/github/aiseongjun/Hands-On-Machine-Learning/blob/main/7_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **7. 앙상블 학습과 랜덤 포레스트**

**앙상블 학습이란?**
```
앙상블 학습은 여러 개의 모델을 결합하여 하나의 강력한 예측 모델을 만드는 기법입니다.
각 개별 모델이 갖는 약점을 보완하고, 전체적으로 더 나은 예측 성능을 얻기 위해 여러 모델을 사용하는 방식입니다.
앙상블 학습은 보통 약한 학습기를 결합하여 강한 학습기를 만드는 데 사용됩니다(큰 수의 법칙).
```

# **7.1 튜표 기반 분류기**

**직접 튜표 분류기**
```
더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 집계하는 것입니다.
직접 튜표 분류기는 다수결 튜표로 정해지는 분류기로, 가장 많은 표를 얻은 클래스가 앙상블의 예측이 됩니다.
```
**앙상블 모델의 정확도를 향상시키는 방법**
```
앙상블 방법은 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘합니다.
다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것입니다.
```

In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators = [
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
).fit(X_train, y_train)

print('clf.predict:\n {}\n'.format([clf.predict(X_test[:1]) for clf in voting_clf.estimators_]))

for name, clf in voting_clf.named_estimators_.items():
  print(name, '=', clf.score(X_test, y_test))
print('voting_clf.score: {}'.format(voting_clf.score(X_test, y_test)))

clf.predict:
 [array([1]), array([1]), array([0])]

lr = 0.864
rf = 0.896
svc = 0.896
voting_clf.score: 0.912


**간접 튜표 분류기**
```
모든 분류기가 클래스의 확률을 예측할 수 있으면 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있습니다.
간접 튜표 분류기는 이 방식을 사용해서 확률이 높은 튜표에 비중을 더 두기 때문에 직접 튜표 방식보다 성능이 높습니다.
```

In [ ]:
voting_clf.voting='soft'
voting_clf.named_estimators['svc'].probability = True
voting_clf.fit(X_train, y_train)
print('voting_clf.score: {}'.format(voting_clf.score(X_test, y_test)))

voting_clf.score: 0.92


# **7.2 배깅과 페이스팅**

**배깅과 페이스팅**
```
다양한 분류기를 만드는 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 랜덤으로 구성하여 분류기를 각기 다르게 학습시키는 것입니다.
훈련 세트에서 중복을 허용하여 샘플링하는 방식을 배깅, 중복을 허용하지 않는 방식을 페이스팅이라고 합니다.
```
**집계 함수**
```
집계 함수는 일반적으로 분류일 때는 통계적 최빈값을, 회귀에 대해서는 평균을 계산합니다.
일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듭니다.
```

## **7.2.1 사이킷런의 배깅과 페이스팅**

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, n_jobs=-1, random_state=42).fit(X_train, y_train)

## **7.2.2 OOB 평가**

**OOB(out-of-bag)**
```
배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있습니다.
선택되지 않은 나머지를 OOB 샘플이라고 부릅니다.
예측기가 훈련되는 동안에는 OOB 샘플을 사용하지 않으므로 OOB 샘플을 사용해 평가할 수 있습니다.
```

In [ ]:
from sklearn.metrics import accuracy_score

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            oob_score=True, n_jobs=1, random_state=42).fit(X_train, y_train)
print('bag_clf.oob_score_: {}'.format(bag_clf.oob_score_))

y_pred = bag_clf.predict(X_test)
print('accuracy_score: {}'.format(accuracy_score(y_test, y_pred)))

print('\nbag_clf.oob_decision_function_[:3]:\n {}'.format(bag_clf.oob_decision_function_[:3]))

bag_clf.oob_score_: 0.896
accuracy_score: 0.92

bag_clf.oob_decision_function_[:3]:
 [[0.32352941 0.67647059]
 [0.3375     0.6625    ]
 [1.         0.        ]]


# **7.3 랜덤 패치와 랜덤 서브스페이스**

**랜덤 패치와 랜덤 서브스페이스**
```
BaggingClassifier는 특성 샘플링도 지원합니다.
각 예측기는 랜덤으로 선택한 입력 특성의 일부분으로 훈련됩니다.
훈련 특성과 샘플을 모두 샘플링하는 것을 랜덤 패치 방식이라 하고
훈련 샘플을 모두 사용하고 특성을 샘플링하는 것을 랜덤 서브스페이스 방식이라고 합니다.
이 기법은 훈련 속도를 크게 높일 수 있어 고차원의 데이터셋을 다룰 때 유용합니다.
```

# **7.4 랜덤 포레스트**

**랜덤 포레스트**
```
랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블입니다.
```

In [ ]:
from sklearn.ensemble import RandomForestClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='sqrt', max_leaf_nodes=16),
    n_estimators=500, n_jobs=-1, random_state=42)

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                 n_jobs=-1, random_state=42).fit(X_train, y_train)
print('rnd_clf.predict:\n {}'.format(rnd_clf.predict(X_test)))

rnd_clf.predict:
 [0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0
 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1
 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 1
 0 0 1 1 0 0 0 0 1 1 1 0 0 0]


## **7.4.1 엑스트라 트리**

**익스트림 랜덤 트리(엑스트라 트리)**
```
트리를 더욱 랜덤하게 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 랜덤으로 분할한 다음 그중에서 최상의 분할을 선택합니다.
이와 같이 극단적으로 랜덤한 트리의 랜덤 포레스트를 익스트림 랜덤 트리 앙상블이라고 부릅니다.
```

## **7.4.2 특성 중요도**

**특성 중요도**
```
사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정합니다.
더 정확하게는 가중치 평균이며, 각 노드의 가중치는 연관된 훈련 샘플 수와 같습니다.
사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화합니다.
```

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42).fit(iris.data, iris.target)
for score, name in zip(rnd_clf.feature_importances_, iris.data.columns):
  print(round(score, 2), name)

0.11 sepal length (cm)
0.02 sepal width (cm)
0.44 petal length (cm)
0.42 petal width (cm)


# **7.5 부스팅**

**부스팅**
```
부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말합니다.
부스팅 방법의 아이디어는 앞의 모델을 보완해 나가면서 일련의 예측기를 학습시키는 것입니다.
```

## **7.5.1 AdaBoost**

**AdaBoost**
```
이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것입니다.
이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 됩니다.
이것이 AdaBoost에서 사용하는 방식입니다.
```
[식 7-4] AdaBoost 예측
$$\hat{y}(X) = \underset{k}{argmax}\sum^{N}_{j=1}\alpha_{j} (\hat{y}_j(X) = k)$$
* $N$은 예측기 수

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=30,
    learning_rate=0.5, random_state=42).fit(X_train, y_train)

## **7.5.2 그레이디언트 부스팅**

**그레이디언트 부스팅**
```
그레이디언트 부스팅은 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가합니다.
하지만 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킵니다.
```

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42).fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43).fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44).fit(X, y3)

X_new = np.array([[-0.4], [0.], [0.5]])
print('tree.predict(X_new):\n {}'.format(sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))))

tree.predict(X_new):
 [0.49484029 0.04021166 0.75026781]


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt_best = GradientBoostingRegressor(
    max_depth=2, learning_rate=0.05, n_estimators=500,
    n_iter_no_change=10, random_state=42).fit(X, y)

print('gbrt_best.n_estimators_: {}'.format(gbrt_best.n_estimators_))

gbrt_best.n_estimators_: 92


## **7.5.3 히스토그램 기반 그레이디언트 부스팅**

**히스토그램 기반 그레이디언트 부스팅**
```
히스토그램 기반 그레이디언트 부스팅은 입력 특성을 구간으로 나누어 정수로 대체하는 방식으로 작동합니다.
```
**GradientBoostingClassifier와의 차이점**
* 인스턴스 수가 10,000개보다 많으면 조기 종료가 자동으로 활성화됩니다. early_stopping 매개변수를 True 또는 False로 설정하여 조기 종료를 항상 켜거나 끌 수 있습니다.
* subsample 매개변수가 지원되지 않습니다.
* n_estimators 매개변수가 max_iter로 바뀌었습니다.
* 조정할 수 있는 결정 트리 하이퍼파라미터는 max_leaf_nodes, min_samples_leaf, max_depth뿐입니다.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgb_reg = make_pipeline(OrdinalEncoder(),
                        HistGradientBoostingRegressor(categorical_features=[0], random_state=42)).fit(X, y)

# **7.6 스태킹**

**스태킹**
```
스태킹은 '앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까?'라는 아이디어에서 출발합니다.
마지막 예측기를 블렌더 또는 메타 학습기라고 합니다.
```

In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator = RandomForestClassifier(random_state=43),
    cv=5
).fit(X_train, y_train)